## Face detection

La ProCam s.p.a ha intenzione di lanciare sul mercato una nuova fotocamera digitale compatta ed economica destinata a piccoli fotografi in erba.

Vieni assunto come Data Scientist per realizzare il sistema di identificazione dei volti nelle immagini, questo permetterà poi ai tecnici della fotografia di ottimizzare le impostazioni per un selfie con una o più persone.

Si tratta di un problema di computer vision, più precisamente di Face Detection.

Devi fornire una pipeline scikit-learn che prende un'immagine in ingresso e ritorna una lista con le coordinate dei bounding box dove sono presenti dei volti, se nell'immagine non contiene volti la lista sarà ovviamente vuota.

- Non ti viene fornito un dataset, sta a te cercarne uno in rete o, nella peggiore delle ipotesi, costruirlo, per semplicità non considereremo implicazioni sulle licenze ad utilizzo commerciale, si tratta pur sempre di un progetto didattico.
- Non puoi utilizzare modelli pre-addestrati, devi addestrarlo tu utilizzando scikit-learn.
- Stai lavorando su un sistema con ridotte capacità di calcolo, quindi il modello deve richiedere poche risorse di calcolo.
- Ovviamente non ti vengono fornite indicazioni sull'implementazione, fai un'approfondita ricerca bibliografica per trovare la soluzione migliore da adottare, il notebook che consegnerai deve essere ben documentato, devi spiegare quali soluzioni hai adottato e perché ed ogni risorsa esterna (paper, blog post, codice github...) che hai utilizzato.
- Il progetto è abbastanza complesso, ricorda che in caso ne avessi necessità puoi sempre chiedere aiuto ai tuoi coach nella Classe Virtuale di Machine Learning su Discord.

https://www.analyticsvidhya.com/blog/2019/09/feature-engineering-images-introduction-hog-feature-descriptor/

https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html

In [1]:
#Import some modules
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage.transform import resize
from skimage.io import imread
from skimage.exposure import rescale_intensity
from sklearn.datasets import fetch_lfw_people


Prima di tutto provo a leggere le immagini di esempio di sklearn e provo ad estrarre hog features da questo... Il resto dopo

In [2]:
#impoort first dataset (faces)
lfw_people = fetch_lfw_people(resize=1)

images_positive = lfw_people.images
#use only a portion of these images, to don't unbalance final dataset
# indexes = np.random.choice(len(lfw_people.images),1600, replace=False)
# images_positive = lfw_people.images[indexes]

#some helper variables
#size = 64 x 128 as original paper
resize_shape = (128, 64) #as row x columns (h x w)

#init list to hold features arrays and labels
features_list = []
labels_list = []

In [3]:
type(lfw_people.images)

numpy.ndarray

In [4]:
#Custom function to process an image
def process_image(img, label):
    img_resized = resize(img, resize_shape)
    
    #feature extraction
    #for hog features use same parametes as in original paper
    fd, hog_image = hog(img_resized, orientations=9, pixels_per_cell=(8, 8), 
                        cells_per_block=(2, 2), visualize=True, feature_vector=True)
    
    #append feature array and labels to helper lists
    features_list.append(fd)
    labels_list.append(label)

In [5]:
#process positive images
for img in tqdm(images_positive, desc="Processing positive images:", 
                unit="item"):
    process_image(img, 1)

Processing positive images:: 100%|██████████| 13233/13233 [04:00<00:00, 55.13item/s]


In [6]:
print(images_positive.dtype)

float32


Import object images from kaggle

In [7]:
# !pip install kaggle

Butterfly dataset (832 images). 

It contains bufferflies images with some flowers and plants. With this dataset model can learn to detect naturalistic objects.

In [8]:
!kaggle datasets download veeralakrishna/butterfly-dataset

Dataset URL: https://www.kaggle.com/datasets/veeralakrishna/butterfly-dataset
License(s): Community Data License Agreement - Permissive - Version 1.0
butterfly-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
# !unzip butterfly-dataset.zip -d butterfly-dataset
!tar -xf butterfly-dataset.zip

Background dataset (715 images).

It contains some backgournd images, taken from streets and landscape photos. With this dataset model can learn to detect common background objects and patterns.

In [10]:
!mkdir stanford-background-dataset

Sottodirectory o file stanford-background-dataset già esistente.


In [11]:
!kaggle datasets download -p stanford-background-dataset balraj98/stanford-background-dataset 

Dataset URL: https://www.kaggle.com/datasets/balraj98/stanford-background-dataset
License(s): other
stanford-background-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [12]:
# !unzip stanford-background-dataset.zip -d stanford-background-dataset
!tar -xf stanford-background-dataset/stanford-background-dataset.zip -C stanford-background-dataset

Add some animals pcitures :)

In [13]:
!mkdir animals

Sottodirectory o file animals già esistente.


In [14]:
!kaggle datasets download -p animals alessiocorrado99/animals10

Dataset URL: https://www.kaggle.com/datasets/alessiocorrado99/animals10
License(s): GPL-2.0
animals10.zip: Skipping, found more recently modified local copy (use --force to force download)


In [15]:
!tar -xf animals/animals10.zip -C animals

In [16]:
#process negative images
butterflies_img_dir = 'leedsbutterfly/images'
background_img_dir = 'stanford-background-dataset/images'
cat_img_dir = 'animals/raw-img/gatto'
chicken_img_dir = 'animals/raw-img/gallina'
cow_img_dir = 'animals/raw-img/mucca'
squirrel_img_dir = 'animals/raw-img/scoiattolo'
sheep_img_dir = 'animals/raw-img/pecora'
negative_img_dirs = [butterflies_img_dir,
                     background_img_dir,
                     cat_img_dir,
                     chicken_img_dir,
                     cow_img_dir,
                     squirrel_img_dir,
                     sheep_img_dir                    
                    ]

# for directory in tqdm(negative_img_dirs,desc="Dataset:",unit="item"):
for directory in negative_img_dirs:
    for filename in tqdm(os.listdir(directory),desc=f"Processing negative images ({directory})",
                        unit="item"):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
            img_path = os.path.join(directory, filename)
            img = imread(img_path, as_gray=True)

            #image normalization in [0,1]
            img_normalized = rescale_intensity(img, in_range='dtype', out_range=(0, 1))

            process_image(img_normalized, 0)

Processing negative images (leedsbutterfly/images): 100%|██████████| 832/832 [01:18<00:00, 10.64item/s]
Processing negative images (stanford-background-dataset/images): 100%|██████████| 715/715 [00:26<00:00, 26.85item/s]
Processing negative images (animals/raw-img/gatto): 100%|██████████| 1668/1668 [03:22<00:00,  8.22item/s]
Processing negative images (animals/raw-img/gallina): 100%|██████████| 3098/3098 [01:36<00:00, 31.99item/s]
Processing negative images (animals/raw-img/mucca): 100%|██████████| 1866/1866 [00:57<00:00, 32.20item/s]
Processing negative images (animals/raw-img/scoiattolo): 100%|██████████| 1862/1862 [01:01<00:00, 30.48item/s]
Processing negative images (animals/raw-img/pecora): 100%|██████████| 1820/1820 [01:16<00:00, 23.80item/s]


In [17]:
#Convert in numpy variables
features_matrix = np.array(features_list)
labels_array = np.array(labels_list).reshape(-1, 1)

#this is the feature matrix, i.e. dataset for classification model
final_matrix = np.hstack((features_matrix, labels_array))

#check
print(final_matrix.shape)

(25094, 3781)


## Classification model (pipeline)